# 8.1 改变对象的字符串显示

In [1]:
class Pair:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Pair({0.x!r},{0.y!r})'.format(self)
    
    def __str__(self):
        return '({0.x!s},{0.y!s})'.format(self)

In [2]:
p = Pair(3,4)

In [3]:
p

Pair(3,4)

In [4]:
print(p)

(3,4)


# 8.2 自定义字符串的格式化

In [5]:
_formats = {
    'ymd':'{d.year}-{d.month}-{d.day}',
    'mdy':'{d.month}/{d.day}/{d.year}',
    'dmy':'{d.day}/{d.month}/{d.year}'
}

class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self,code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [6]:
d = Date(2019,1,1)

In [7]:
format(d)

'2019-1-1'

In [8]:
format(d,'mdy')

'1/1/2019'

In [9]:
'The date is {:ymd}'.format(d)

'The date is 2019-1-1'

In [10]:
'The date is {:mdy}'.format(d)

'The date is 1/1/2019'

In [11]:
from datetime import date

In [12]:
d = date(2012, 12, 21)

In [14]:
format(d)

'2012-12-21'

In [15]:
format(d,'%A, %B %d, %Y')

'Friday, December 21, 2012'

# 8.3 让对象支持上下文管理协议

为了让一个对象兼容 with 语句，你需要实现 __enter__() 和 __exit__() 方法。 例如，考虑如下的一个类，它能为我们创建一个网络连接：

In [16]:
from socket import socket,AF_INET,SOCK_STREAM

In [22]:
class LazyConnection:
    def __init__(self,address,family=AF_INET,type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None
        
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family,self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self,exc_ty,exc_val,tb):
        self.sock.close()
        self.sock = None

In [23]:
from functools import partial

In [24]:
conn = LazyConnection(('www.python.org',80))
with conn as s:
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv,8192),b''))

In [26]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))
with conn as s1:
    pass
    with conn as s2:
        pass
        # s1 and s2 are independent sockets

# 8.4 创建大量对象时节省内存方法

对于主要是用来当成简单的数据结构的类而言，你可以通过给类添加 __slots__ 属性来极大的减少实例所占的内存。

In [27]:
class Date:
    __slots__ = ['year','month','day']
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day

# 8.5 在类中封装属性名

Python程序员不去依赖语言特性去封装数据，而是通过遵循一定的属性和方法命名规约来达到这个效果。 第一个约定是任何以单下划线_开头的名字都应该是内部实现

In [28]:
class A:
    def __init__(self):
        self._internal = 0
        self.public = 1
        
    def public_method(self):
        pass
    
    def _internal_method(self):
        pass

使用双下划线开始会导致访问名称变成其他形式。 比如，在前面的类B中，私有属性会被分别重命名为 _B__private 和 _B__private_method 。 这时候你可能会问这样重命名的目的是什么，答案就是继承——这种属性通过继承是无法被覆盖的。

In [29]:
class B:
    def __init__(self):
        self.__private = 0

    def __private_method(self):
        pass

    def public_method(self):
        pass
        self.__private_method()

In [30]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private

    # Does not override B.__private_method()
    def __private_method(self):
        pass

这里，私有名称 \__private 和 __private_method 被重命名为 _C__private 和 _C__private_method ，这个跟父类B中的名称是完全不同的。

# 8.6 创建可管理的属性

自定义某个属性的一种简单方法是将它定义为一个property。

In [37]:
class Person:
    def __init__(self,first_name):
        self.first_name = first_name
        
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self,value):
        if not isinstance(value,str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

上述代码中有三个相关联的方法，这三个方法的名字都必须一样。 第一个方法是一个 getter 函数，它使得 first_name 成为一个属性。 其他两个方法给 first_name 属性添加了 setter 和 deleter 函数。 需要强调的是只有在 first_name 属性被创建后， 后面的两个装饰器 @first_name.setter 和 @first_name.deleter 才能被定义。

property的一个关键特征是它看上去跟普通的attribute没什么两样， 但是访问它的时候会自动触发 getter 、setter 和 deleter 方法。

In [33]:
a = Person('Guido')

In [34]:
a.first_name

'Guido'

In [38]:
a.first_name = 43

TypeError: Expected a string

In [39]:
del a.first_name

NameError: name 'AttrbuteError' is not defined

在实现一个property的时候，底层数据(如果有的话)仍然需要存储在某个地方。 因此，在get和set方法中，你会看到对 _first_name 属性的操作，这也是实际数据保存的地方。 另外，你可能还会问为什么 __init__() 方法中设置了 self.first_name 而不是 self._first_name 。 在这个例子中，我们创建一个property的目的就是在设置attribute的时候进行检查。 因此，你可能想在初始化的时候也进行这种类型检查。通过设置 self.first_name ，自动调用 setter 方法， 这个方法里面会进行参数的检查，否则就是直接访问 self._first_name 了。

In [40]:
class Person:
    def __init__(self,first_name):
        self.set_first_name(first_name)
        
    def get_first_name(self):
        return self._first_name
    
    def set_first_name(self,value):
        if not isinstance(value,str):
            raise TypeError('Expceted a string')
        self._first_name = value
        
    def del_first_name(self):
        raise AttributeError("Can't delete attribute")
        
    name = property(get_first_name,set_first_name,del_first_name)

Properties还是一种定义动态计算attribute的方法。 这种类型的attributes并不会被实际的存储，而是在需要的时候计算出来。

In [41]:
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property
    def area(self):
        return math.pi * self.radius ** 2

    @property
    def diameter(self):
        return self.radius * 2

    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

In [42]:
c = Circle(4.0)

In [43]:
c.radius

4.0

In [44]:
c.area

50.26548245743669

In [45]:
c.diameter

8.0

In [46]:
p = Person('Gaudi')

In [47]:
p.get_first_name()

'Gaudi'

In [48]:
p.set_first_name('Larry')

# 8.7 调用父类方法

In [49]:
class A:
    def spam(self):
        print('A.spam')
        
class B(A):
    def spam(self):
        print('B.spam')
        super().spam()

In [51]:
b = B()

In [52]:
b.spam()

B.spam
A.spam


In [53]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [54]:
b = B()

In [55]:
b.x

0

super() 的另外一个常见用法出现在覆盖Python特殊方法的代码中

In [57]:
class Proxy:
    def __init__(self,obj):
        self._obj = obj
        
    def __getattr__(self,name):
        return getattr(self._obj,name)
    
    def __setattr__(self,name,value):
        if name.startwith('_'):
            super().__setarrt__(name,value)
        else:
            setattr(self._obj,name,value)

In [58]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A,B):
    def __init__(self):
        super().__init__()  # Only one call to super() here
        print('C.__init__')

In [59]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


 对于你定义的每一个类，Python会计算出一个所谓的方法解析顺序(MRO)列表。 这个MRO列表就是一个简单的所有基类的线性顺序表。

In [60]:
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

而这个MRO列表的构造是通过一个C3线性化算法来实现的。 我们不去深究这个算法的数学原理，它实际上就是合并所有父类的MRO列表并遵循如下三条准则：

- 子类会先于父类被检查
- 多个父类会根据它们在列表中的顺序被检查
- 如果对下一个类存在两个合法的选择，选择第一个父类

老实说，你所要知道的就是MRO列表中的类顺序会让你定义的任意类层级关系变得有意义。

当你使用 super() 函数时，Python会在MRO列表上继续搜索下一个类。 只要每个重定义的方法统一使用 super() 并只调用它一次， 那么控制流最终会遍历完整个MRO列表，每个方法也只会被调用一次。 这也是为什么在第二个例子中你不会调用两次 Base.__init__() 的原因。

super() 有个令人吃惊的地方是它并不一定去查找某个类在MRO中下一个直接父类， 你甚至可以在一个没有直接父类的类中使用它。

In [61]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

In [62]:
a = A()
a.spam()

A.spam


AttributeError: 'super' object has no attribute 'spam'

In [63]:
class B:
     def spam(self):
         print('B.spam')

In [64]:
class C(A,B):
     pass

In [65]:
c = C()
c.spam()

A.spam
B.spam


你可以看到在类A中使用 super().spam() 实际上调用的是跟类A毫无关系的类B中的 spam() 方法。这个用类C的MRO列表就可以完全解释清楚了 

然而，由于 super() 可能会调用不是你想要的方法，你应该遵循一些通用原则。 首先，确保在继承体系中所有相同名字的方法拥有可兼容的参数签名(比如相同的参数个数和参数名称)。 这样可以确保 super() 调用一个非直接父类方法时不会出错。 其次，最好确保最顶层的类提供了这个方法的实现，这样的话在MRO上面的查找链肯定可以找到某个确定的方法。

# 8.8 子类中扩展property

In [66]:
class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

下面是一个示例类，它继承自Person并扩展了 name 属性的功能

In [71]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self,value):
        print('Setting name to',value)
        super(SubPerson,SubPerson).name.__set__(self,value)
        
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson,SubPerson).name.__delete__(self)

In [72]:
s = SubPerson('Guido')

Setting name to Guido


In [73]:
s.name

Getting name


'Guido'

In [74]:
s.name = 'Larry'

Setting name to Larry


In [75]:
s.name = 42

Setting name to 42


TypeError: Expected a string

如果你仅仅只想扩展property的某一个方法，那么可以像下面这样写：

In [76]:
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name

In [77]:
t = SubPerson('Guido')

In [78]:
t.name

Getting name


'Guido'

In [79]:
t.name = 'ggg'

In [80]:
t.name

Getting name


'ggg'

或者，你只想修改setter方法，就这么写：

In [81]:
class SubPerson(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

在子类中扩展一个property可能会引起很多不易察觉的问题， 因为一个property其实是 getter、setter 和 deleter 方法的集合，而不是单个方法。 因此，当你扩展一个property的时候，你需要先确定你是否要重新定义所有的方法还是说只修改其中某一个。

在第一个例子中，所有的property方法都被重新定义。 在每一个方法中，使用了 super() 来调用父类的实现。 在 setter 函数中使用 super(SubPerson, SubPerson).name.__set__(self, value) 的语句是没有错的。 为了委托给之前定义的setter方法，需要将控制权传递给之前定义的name属性的 __set__() 方法。 不过，获取这个方法的唯一途径是使用类变量而不是实例变量来访问它。 这也是为什么我们要使用 super(SubPerson, SubPerson) 的原因。

如果你只想重定义其中一个方法，那只使用 @property 本身是不够的。比如，下面的代码就无法工作：

In [82]:
class SubPerson(Person):
    @property  # Doesn't work
    def name(self):
        print('Getting name')
        return super().name

In [83]:
s = SubPerson('Guido')

AttributeError: can't set attribute

你应该像之前说过的那样修改代码：

In [84]:
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name

在这个特别的解决方案中，我们没办法使用更加通用的方式去替换硬编码的 Person 类名。 如果你不知道到底是哪个基类定义了property， 那你只能通过重新定义所有property并使用 super() 来将控制权传递给前面的实现。

值得注意的是上面演示的第一种技术还可以被用来扩展一个描述器(在8.9小节我们有专门的介绍)。比如：

In [85]:
class String:
    def __init__(self,name):
        self.name = name
    
    def __get__(self,instance,cls):
        if instance is None:
            return self
        return instance.__dict__[self.name]
    
    def __set__(self,instance,value):
        if not isinstance(value,str):
            raise TypeError('Expected a string')
        instance.__dict__[self.name] = value
        
class Person:
    name = String('name')
    
    def __init__(self,name):
        self.name = name
        
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self):
        print('Setting name to',value)
        super(SubPerson,SubPerson).name.__set__(self,value)
        
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson,SubPerson).name.__delete__(self)

# 8.9 创建新的类或实例属性

如果你想创建一个全新的实例属性，可以通过一个描述器类的形式来定义它的功能

In [89]:
class Integer:
    def __init__(self,name):
        self.name = name
        
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self,instance,value):
        if not isinstance(value,int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value
        
    def __delete__(self,instance):
        del instance.__dict__[self.name]

In [90]:
class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self,x,y):
        self.x = x
        self.y = y

In [91]:
p = Point(2,3)

In [92]:
p.x

2

In [93]:
p.y

3

In [94]:
p.x = 2.3

TypeError: Expected an int

In [95]:
Point.x

In [100]:
class Typed:
    def __init__(self,name,expected_type):
        self.name = name
        self.expected_type = expected_type
        
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self,instance,value):
        if not isinstance(value,self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        instance.__dict__[self.name] = value  
    def __delete__(self,instance):
        del instance.__dict__[self.name]
        


In [101]:
def typeassert(**kwargs):
    def decorate(cls):
        for name,expected_type in kwargs.items():
            setattr(cls,name,Typed(name,expected_type))
        return cls
    return decorate

In [102]:
@typeassert(name=str,shares=int,price=float)
class Stock:
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price

In [103]:
s = Stock('gudd',23,12.3)

In [104]:
t = Stock('gga','34',12)

TypeError: Expected <class 'int'>

# 8.10 使用延迟计算属性

定义一个延迟属性的一种高效方法是通过使用一个描述器类

In [105]:
class lazyproperty:
    def __init__(self,func):
        self.func = func
        
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance,self.func.__name__,value)
            return value
            

In [106]:
import math

class Circle:
    def __init__(self,radius):
        self.radius = radius
        
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi*self.radius**2
    
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2*math.pi*self.radius

In [107]:
c = Circle(4.0)

In [108]:
c.radius

4.0

In [109]:
c.area

Computing area


50.26548245743669

In [110]:
c.area

50.26548245743669

In [111]:
c = Circle(4.0)

In [112]:
vars(c)

{'radius': 4.0}

In [113]:
c.area

Computing area


50.26548245743669

In [114]:
vars(c)

{'radius': 4.0, 'area': 50.26548245743669}

In [115]:
c.area

50.26548245743669

In [116]:
del c.area

In [117]:
vars(c)

{'radius': 4.0}

In [118]:
c.area

Computing area


50.26548245743669

In [119]:
c.area = 25

In [120]:
c.area

25

# 8.11 简化数据结构的初始化

可以在一个基类中写一个公用的 __init__() 函数

In [121]:
import math

class Structure1:
    _fields = []
    
    def __init__(self,*args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        for name, value in zip(self._fields,args):
            setattr(self,name,value)

In [123]:
class Stock(Structure1):
    _fields = ['name','shares','price']
    
class Point(Structure1):
    _fields = ['x','y']

In [124]:
s = Stock('Acme',50,91.1)

In [125]:
p = Point(2,3)

In [126]:
s2 = Stock('Ame',3)

TypeError: Expected 3 arguments

如果还想支持关键字参数，可以将关键字参数设置为实例属性

In [127]:
class Stucture2:
    _fields = []

    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # Set all of the positional arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        
        for name in self._fields[len(args):]:
            setattr(self,name,kwargs.pop(name))
        
        if kwargs:
            raise TypeError('Invalid argument(s): {}'.format(','.join(kwargs)))

In [128]:
class Stock(Stucture2):
    _fields = ['name','shares','price']
    
s1 = Stock('ACME',50,91)
s2 = Stock('ACME',50,price=91)

In [129]:
s3 = Stock('ACME', shares=50, price=91.1, aa=1)

TypeError: Invalid argument(s): aa

In [130]:
class Structure3:
    _fields = []
    
    def __init__(self,*args,**kwargs):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
            
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self,name,kwargs.pop(name))
            
        if kwargs:
            raise TypeError('Duplicate values for {}'.format(','.join(kwargs)))
            

In [131]:
class Stock(Structure3):
        _fields = ['name', 'shares', 'price']


In [132]:
s1 = Stock('ACME', 50, 91.1)

In [133]:
s2 = Stock('ACME', 50, 91.1, date='8/2/2012')

In [134]:
_fields = ['name', 'shares', 'price']

In [136]:
kwargs = {date:'8/2/2012'}

In [137]:
extra_args = kwargs.keys() - _fields

In [138]:
extra_args

{datetime.date}

In [139]:
kwargs.keys()

dict_keys([<class 'datetime.date'>])

# 8.12 定义接口或者抽象基类

使用 abc 模块可以很轻松的定义抽象基类

In [140]:
from abc import ABCMeta,abstractmethod

In [141]:
class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self,maxbytes=-1):
        pass
    
    @abstractmethod
    def wirte(self,data):
        pass

抽象类的一个特点是它不能直接被实例化,抽象类的目的就是让别的类继承它并实现特定的抽象方法。

In [142]:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass

    def write(self, data):
        pass

抽象基类的一个主要用途是在代码中检查某些类是否为特定类型，实现了特定接口：

In [143]:
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    pass

除了继承这种方式外，还可以通过注册方式来让某个类实现抽象基类

In [145]:
import io

In [146]:
IStream.register(io.IOBase)

io.IOBase

In [147]:
f = open('foo.txt')

In [149]:
isinstance(f,IStream)

True

@abstractmethod 还能注解静态方法、类方法和 properties 。 你只需保证这个注解紧靠在函数定义前即可

In [150]:
class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass

    @name.setter
    @abstractmethod
    def name(self, value):
        pass

    @classmethod
    @abstractmethod
    def method1(cls):
        pass

    @staticmethod
    @abstractmethod
    def method2():
        pass

标准库中有很多用到抽象基类的地方。collections 模块定义了很多跟容器和迭代器(序列、映射、集合等)有关的抽象基类。 numbers 库定义了跟数字对象(整数、浮点数、有理数等)有关的基类。io 库定义了很多跟I/O操作相关的基类。

你可以使用预定义的抽象类来执行更通用的类型检查

import collections

\# Check if x is a sequence

if isinstance(x, collections.Sequence):

...

\# Check if x is iterable

if isinstance(x, collections.Iterable):

...

\# Check if x has a size

if isinstance(x, collections.Sized):

...

\# Check if x is a mapping

if isinstance(x, collections.Mapping):

尽管ABCs可以让我们很方便的做类型检查，但是我们在代码中最好不要过多的使用它。 因为Python的本质是一门动态编程语言，其目的就是给你更多灵活性， 强制类型检查或让你代码变得更复杂，这样做无异于舍本求末。

# 8.13 实现数据模型的类型约束

在这个问题中，你需要在对某些实例属性赋值时进行检查。 所以你要自定义属性赋值函数，这种情况下最好使用描述器。

In [151]:
class Descriptor:
    def __init__(self,name=None,**opts):
        self.name = name
        for key,value in opts.items():
            setattr(self,key,value)
        
    def __set__(self,instance,value):
        instance.__dict__[self.name] = value
        
class Typed(Descriptor):
    expected_type = type(None)
    
    def __set__(self,instance,value):
        if not isinstance(value,self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance,value)
        
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)


class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)

In [154]:
class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass

In [155]:
class Stock:
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price

In [156]:
s = Stock('Acme',75,15.0)

In [157]:
s.name

'Acme'

In [158]:
s.shares

75

In [159]:
s.shares = -10

ValueError: Expected >= 0

In [160]:
s.price='a lot'

TypeError: expected <class 'float'>

In [167]:
def check_attributes(**kwargs):
    def decorate(cls):
        for key,value in kwargs.items():
            if instance(value,Descriptor):
                value.name = key
                setattr(cls,key,value)
            else:
                setattr(cls,key,value(key))
                
@check_attributes(name=SizedString(size=8),
                  shares=UnsignedInteger,
                  price=UnsignedFloat)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

TypeError: 'NoneType' object is not callable

In [168]:
# A metaclass that applies checking
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        # Attach attribute names to the descriptors
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
        return type.__new__(cls, clsname, bases, methods)

# Example
class Stock2(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [169]:
# Decorator for applying type checking
def Typed(expected_type, cls=None):
    if cls is None:
        return lambda cls: Typed(expected_type, cls)
    super_set = cls.__set__

    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError('expected ' + str(expected_type))
        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


# Decorator for unsigned values
def Unsigned(cls):
    super_set = cls.__set__

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


# Decorator for allowing sized values
def MaxSized(cls):
    super_init = cls.__init__

    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super_init(self, name, **opts)

    cls.__init__ = __init__

    super_set = cls.__set__

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super_set(self, instance, value)

    cls.__set__ = __set__
    return cls


# Specialized descriptors
@Typed(int)
class Integer(Descriptor):
    pass


@Unsigned
class UnsignedInteger(Integer):
    pass


@Typed(float)
class Float(Descriptor):
    pass


@Unsigned
class UnsignedFloat(Float):
    pass


@Typed(str)
class String(Descriptor):
    pass


@MaxSized
class SizedString(String):
    pass

# 8.14 实现自定义容器

In [170]:
import collections

In [174]:
import bisect
class SortedItems(collections.Sequence):
    def __init__(self,initial=None):
        self._items = sorted(initial) if initial is not None else []
        
    def __getitem__(self,index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    def add(self,item):
        bisect.insort(self._items,item)

In [175]:
s = SortedItems([1,4,2,9,7])

In [176]:
list(s)

[1, 2, 4, 7, 9]

In [177]:
s[0]

1

In [178]:
s[1]

2

In [179]:
s.add(5)

In [180]:
s

In [181]:
list(s)

[1, 2, 4, 5, 7, 9]

In [182]:
len(s)

6

In [183]:
s.__len__()

6

In [184]:
class Items(collections.MutableSequence):
    def __init__(self, initial=None):
        self._items = list(initial) if initial is not None else []

    # Required sequence methods
    def __getitem__(self, index):
        print('Getting:', index)
        return self._items[index]

    def __setitem__(self, index, value):
        print('Setting:', index, value)
        self._items[index] = value

    def __delitem__(self, index):
        print('Deleting:', index)
        del self._items[index]

    def insert(self, index, value):
        print('Inserting:', index, value)
        self._items.insert(index, value)

    def __len__(self):
        print('Len')
        return len(self._items)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In [185]:
a = Items([1,2,3])

In [186]:
len(a)

Len


3

In [187]:
a.append(4)

Len
Inserting: 3 4


In [188]:
a.append(2)

Len
Inserting: 4 2


In [189]:
a.count(2)

Getting: 0
Getting: 1
Getting: 2
Getting: 3
Getting: 4
Getting: 5


2

# 8.15 属性的代理访问

代理是一种编程模式，它将某个操作转移给另外一个对象来实现

In [190]:
class A:
    def spam(self, x):
        pass

    def foo(self):
        pass


class B1:
    """简单的代理"""

    def __init__(self):
        self._a = A()

    def spam(self, x):
        # Delegate to the internal self._a instance
        return self._a.spam(x)

    def foo(self):
        # Delegate to the internal self._a instance
        return self._a.foo()

    def bar(self):
        pass

如果仅仅就两个方法需要代理，那么像这样写就足够了。但是，如果有大量的方法需要代理， 那么使用 __getattr__() 方法或许或更好些

In [191]:
class B2:
    """使用__getattr__的代理，代理方法比较多时候"""

    def __init__(self):
        self._a = A()

    def bar(self):
        pass

    # Expose all of the methods defined on class A
    def __getattr__(self, name):
        """这个方法在访问的attribute不存在的时候被调用
        the __getattr__() method is actually a fallback method
        that only gets called when an attribute is not found"""
        return getattr(self._a, name)

In [192]:
b = B2()

In [193]:
b.bar()

In [194]:
b.spam(42)

In [206]:
class Proxy:
    def __init__(self,obj):
        self._obj = obj
        
    def __getattr__(self,name):
        print('getattr:',name)
        return getattr(self._obj,name)
    
    def __setattr__(self,name,value):
        if name.startswith('_'):
            super().__setattr__(name,value)
        else:
            print('setattr:',name,value)
            setattr(self._obj,name,value)
            
    def __delattr__(self,name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)

In [207]:
class Spam:
    def __init__(self,x):
        self.x = x
        
    def bar(self,y):
        print('Spam.bar:',self.x,y)

In [208]:
s = Spam(2)

In [209]:
s.x

2

In [210]:
p = Proxy(s)

In [211]:
p.x

getattr: x


2

In [212]:
p.bar(3)

getattr: bar
Spam.bar: 2 3


In [213]:
p.x = 37

setattr: x 37


In [214]:
class A:
    def spam(self,x):
        print('A.spam',x)
    def foo(self):
        print('A.foo')
        
class B(A):
    def spam(self, x):
        print('B.spam')
        super().spam(x)
    def bar(self):
        print('B.bar')

In [215]:
class A:
    def spam(self, x):
        print('A.spam', x)
    def foo(self):
        print('A.foo')
        
class B:
    def __init__(self):
        self._a = A()
    def spam(self, x):
        print('B.spam', x)
        self._a.spam(x)
    def bar(self):
        print('B.bar')
    def __getattr__(self, name):
        return getattr(self._a, name)

当实现代理模式时，还有些细节需要注意。 首先，__getattr__() 实际是一个后备方法，只有在属性不存在时才会调用。 因此，如果代理类实例本身有这个属性的话，那么不会触发这个方法的。 另外，\__setattr__() 和 \__delattr__() 需要额外的魔法来区分代理实例和被代理实例 \_obj 的属性。 一个通常的约定是只代理那些不以下划线 _ 开头的属性(代理类只暴露被代理类的公共属性)。

还有一点需要注意的是，\__getattr__() 对于大部分以双下划线(__)开始和结尾的属性并不适用。 比如，考虑如下的类：

In [216]:
class ListLike:
    """__getattr__对于双下划线开始和结尾的方法是不能用的，需要一个个去重定义"""

    def __init__(self):
        self._items = []

    def __getattr__(self, name):
        return getattr(self._items, name)

In [217]:
a = ListLike()

In [218]:
a.append(2)

In [219]:
a.insert(0,1)

In [220]:
a.sort()

In [221]:
len(a)

TypeError: object of type 'ListLike' has no len()

为了让它支持这些方法，你必须手动的实现这些方法代理：

In [222]:
class ListLike:
    """__getattr__对于双下划线开始和结尾的方法是不能用的，需要一个个去重定义"""

    def __init__(self):
        self._items = []

    def __getattr__(self, name):
        return getattr(self._items, name)

    # Added special methods to support certain list operations
    def __len__(self):
        return len(self._items)

    def __getitem__(self, index):
        return self._items[index]

    def __setitem__(self, index, value):
        self._items[index] = value

    def __delitem__(self, index):
        del self._items[index]

8.16 在类中定义多个构造器

为了实现多个构造器，你需要使用到类方法

In [227]:
import time

class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
        
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year,t.tm_mon,t.tm_mday)

In [228]:
a = Date(2013,3,4)

In [229]:
a

In [230]:
b = Date.today()

# 8.17 创建不调用init方法的实例

可以通过 __new__() 方法创建一个未初始化的实例。

In [231]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

下面演示如何不调用 __init__() 方法来创建这个Date实例：

In [232]:
d = Date.__new__(Date)

In [233]:
d

In [234]:
d.year

AttributeError: 'Date' object has no attribute 'year'

In [235]:
data = {'year':2012, 'month':8, 'day':29}
for key,value in data.items():
    setattr(d,key,value)

In [236]:
d.year

2012

当我们在反序列对象或者实现某个类方法构造函数时需要绕过 __init__() 方法来创建对象。 例如，对于上面的Date来讲，有时候你可能会像下面这样定义一个新的构造函数 today() ：

In [237]:
from time import localtime

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        d = cls.__new__(cls)
        t = localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_mday
        return d

# 8.18 利用Mixins扩展类功能

你有很多有用的方法，想使用它们来扩展其他类的功能。但是这些类并没有任何继承的关系。 因此你不能简单的将这些方法放入一个基类，然后被其他类继承

通常当你想自定义类的时候会碰上这些问题。可能是某个库提供了一些基础类， 你可以利用它们来构造你自己的类。

假设你想扩展映射对象，给它们添加日志、唯一性设置、类型检查等等功能。

In [1]:
class LoggedMappingMixin:
    __slots__ = ()
    
    def __getitem__(self,key):
        print('Getting ' + str(key))
        return super().__getitem__(key)
    
    def __setitem__(self,key,value):
        print('Setting {}={!r}'.format(key,value))
        return super().__setitem__(key,value)
    
    def __delitem__(self,key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)

In [2]:
class SetOnceMappingMixin:
    '''
    Only allow a key to be set once.
    '''
    __slots__ = ()

    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)


In [3]:
class StringKeysMappingMixin:
    '''
    Restrict keys to strings only
    '''
    __slots__ = ()

    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value)

这些类单独使用起来没有任何意义，事实上如果你去实例化任何一个类，除了产生异常外没任何作用。 它们是用来通过多继承来和其他映射对象混入使用的

In [4]:
class LoggedDict(LoggedMappingMixin,dict):
    pass

In [5]:
d = LoggedDict()

In [6]:
d['X'] = 23

Setting X=23


In [7]:
print(d['X'])

Getting X
23


In [9]:
del d['X']

Deleting X


In [10]:
from collections import defaultdict

class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
    pass

In [11]:
d = SetOnceDefaultDict(list)

In [12]:
d['x'].append(2)

In [13]:
d['x'].append(3)

In [14]:
d['x'] = 23

KeyError: 'x already set'

你会经常使用 socketserver 模块中的 ThreadingMixIn 来给其他网络相关类增加多线程支持。 例如，下面是一个多线程的XML-RPC服务：

In [16]:
from xmlrpc.server import SimpleXMLRPCServer
from socketserver import ThreadingMixIn
class ThreadedXMLRPCServer(ThreadingMixIn, SimpleXMLRPCServer):
    pass

同时在一些大型库和框架中也会发现混入类的使用，用途同样是增强已存在的类的功能和一些可选特征。

对于混入类，有几点需要记住。首先是，混入类不能直接被实例化使用。 其次，混入类没有自己的状态信息，也就是说它们并没有定义 \__init__() 方法，并且没有实例属性。 这也是为什么我们在上面明确定义了 \__slots__ = () 。

还有一种实现混入类的方式就是使用类装饰器，

In [17]:
def LoggedMapping(cls):
    """第二种方式：使用类装饰器"""
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__

    def __getitem__(self, key):
        print('Getting ' + str(key))
        return cls_getitem(self, key)

    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return cls_setitem(self, key, value)

    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return cls_delitem(self, key)

    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls


@LoggedMapping
class LoggedDict(dict):
    pass

# 8.19 实现状态对象或者状态机

你想实现一个状态机或者是在不同状态下执行操作的对象，但是又不想在代码中出现太多的条件判断语句。

在很多程序中，有些对象会根据状态的不同来执行不同的操作。比如考虑如下的一个连接对象：

In [18]:
class Connection:
    """普通方案，好多个判断语句，效率低下~~"""

    def __init__(self):
        self.state = 'CLOSED'

    def read(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('reading')

    def write(self, data):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('writing')

    def open(self):
        if self.state == 'OPEN':
            raise RuntimeError('Already open')
        self.state = 'OPEN'

    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Already closed')
        self.state = 'CLOSED'

这样写有很多缺点，首先是代码太复杂了，好多的条件判断。其次是执行效率变低， 因为一些常见的操作比如read()、write()每次执行前都需要执行检查。

一个更好的办法是为每个状态定义一个对象

In [22]:
class Connection:
    
    def __init__(self):
        self.new_state(ClosedConnectionState)
        
    def new_state(self,newstate):
        self._state = newstate
        
    def read(self):
        return self._state.read(self)
    
    def write(self,data):
        return self._state.write(self,data)
    
    def open(self):
        return self._state.open(self)
    
    def close(self):
        return self._state.close(self)
    
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()
        
    @staticmethod
    def write(conn,data):
        raise NotImplementedError()
        
    @staticmethod
    def open(conn):
        raise NotImplementedError()
        
    @staticmethod
    def close(conn):
        raise NotImplementedError()
        
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')
        
    @staticmethod
    def write(conn,data):
        raise RuntimeError('Not open')
        
    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)
        
    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')
        
class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
        
    @staticmethod
    def write(conn,data):
        print('writing')
        
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
        
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)

In [23]:
c = Connection()        

In [24]:
c._state

__main__.ClosedConnectionState

In [25]:
c.read()

RuntimeError: Not open

In [26]:
c.open()

In [27]:
c.read()

reading


In [28]:
c.write('hello')

writing


In [30]:
c.close()

In [31]:
c._state

__main__.ClosedConnectionState

# 8.20 通过字符串调用对象方法

你有一个字符串形式的方法名称，想通过它调用某个对象的对应方法。最简单的情况，可以使用 getattr()

In [32]:
import math

class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Point({!r:},{!r:})'
    
    def distance(self,x,y):
        return math.hypot(self.x - x,self.y - y)

In [33]:
p = Point(2,3)

In [35]:
d = getattr(p,'distance')(0,0)

In [36]:
d

3.605551275463989

另外一种方法是使用 operator.methodcaller() 

In [37]:
import operator

operator.methodcaller('distance',0,0)(p)

3.605551275463989

当你需要通过相同的参数多次调用某个方法时，使用 operator.methodcaller 就很方便了。 比如你需要排序一系列的点，就可以这样做：

In [40]:
points = [
    Point(1, 2),
    Point(3, 0),
    Point(10, -3),
    Point(-5, -7),
    Point(-1, 8),
    Point(3, 2)
]
# Sort by distance from origin (0, 0)
points.sort(key=operator.methodcaller('distance', 0, 0))

In [41]:
points

[Point({!r:},{!r:}),
 Point({!r:},{!r:}),
 Point({!r:},{!r:}),
 Point({!r:},{!r:}),
 Point({!r:},{!r:}),
 Point({!r:},{!r:})]

调用一个方法实际上是两部独立操作，第一步是查找属性，第二步是函数调用。 因此，为了调用某个方法，你可以首先通过 getattr() 来查找到这个属性，然后再去以函数方式调用它即可。

operator.methodcaller() 创建一个可调用对象，并同时提供所有必要参数， 然后调用的时候只需要将实例对象传递给它即可，比如：

In [42]:
p = Point(3, 4)

In [43]:
d = operator.methodcaller('distance', 0, 0)

In [44]:
d(p)

5.0

# 8.21 实现访问者模式

你要处理由大量不同类型的对象组成的复杂数据结构，每一个对象都需要进行不同的处理。 比如，遍历一个树形结构，然后根据每个节点的相应状态执行不同的操作。

这里遇到的问题在编程领域中是很普遍的，有时候会构建一个由大量不同对象组成的数据结构。

In [47]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

In [48]:
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

In [49]:
type(t4).__name__

'Add'

In [50]:
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

In [51]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -node.operand

In [52]:
e = Evaluator()

In [53]:
e.visit(t4)

0.6

In [59]:
class StackCode(NodeVisitor):
    def generate_code(self,node):
        self.instructions = []
        self.visit(node)
        return self.instructions
    
    def visit_Number(self,node):
        self.instructions.append(('PUSH',node.value))
        
    def binop(self,node,instruction):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instruction,))
        
    def visit_Add(self, node):
        self.binop(node, 'ADD')

    def visit_Sub(self, node):
        self.binop(node, 'SUB')

    def visit_Mul(self, node):
        self.binop(node, 'MUL')

    def visit_Div(self, node):
        self.binop(node, 'DIV')
        
    def unaryop(self,node,instruction):
        self.visit(node.operand)
        self.instructions.append((instruction,))
        
    def visit_Negate(self,node):
        self.unaryop(node,'NEG')

In [60]:
s = StackCode()

In [61]:
s.generate_code(t4)

[('PUSH', 1),
 ('PUSH', 2),
 ('PUSH', 3),
 ('PUSH', 4),
 ('SUB',),
 ('MUL',),
 ('PUSH', 5),
 ('DIV',),
 ('ADD',)]

刚开始的时候你可能会写大量的if/else语句来实现， 这里访问者模式的好处就是通过 getattr() 来获取相应的方法，并利用递归来遍历所有的节点

In [62]:
def binop(self, node, instruction):
    self.visit(node.left)
    self.visit(node.right)
    self.instructions.append((instruction,))

还有一点需要指出的是，这种技术也是实现其他语言中switch或case语句的方式。 比如，如果你正在写一个HTTP框架，你可能会写这样一个请求分发的控制器：

In [63]:
class HTTPHandler:
    def handle(self, request):
        methname = 'do_' + request.request_method
        getattr(self, methname)(request)
    def do_GET(self, request):
        pass
    def do_POST(self, request):
        pass
    def do_HEAD(self, request):
        pass

访问者模式一个缺点就是它严重依赖递归，如果数据结构嵌套层次太深可能会有问题， 有时候会超过Python的递归深度限制

# 8.22 不用递归实现访问者模式

你使用访问者模式遍历一个很深的嵌套树形数据结构，并且因为超过嵌套层级限制而失败。 你想消除递归，并同时保持访问者编程模式。

通过巧妙的使用生成器可以在树遍历或搜索算法中消除递归。 在8.21小节中，我们给出了一个访问者类。下面我们利用一个栈和生成器重新实现这个类：

In [69]:
import types

class Node:
    pass

class NodeVisitor:
    def visit(self, node):
        stack = [node]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()

        return last_result

    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

In [70]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

# A sample visitor class that evaluates expressions
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -self.visit(node.operand)

if __name__ == '__main__':
    # 1 + 2*(3-4) / 5
    t1 = Sub(Number(3), Number(4))
    t2 = Mul(Number(2), t1)
    t3 = Div(t2, Number(5))
    t4 = Add(Number(1), t3)
    # Evaluate it
    e = Evaluator()
    print(e.visit(t4))  # Outputs 0.6

0.6


如果嵌套层次太深那么上述的Evaluator就会失效：

In [71]:
a = Number(0)
for n in range(1, 100000):
    a = Add(a, Number(n))
e = Evaluator()
e.visit(a)

RecursionError: maximum recursion depth exceeded while calling a Python object

现在我们稍微修改下上面的Evaluator：

In [72]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)

    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)

    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)

    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)

    def visit_Negate(self, node):
        yield - (yield node.operand)

In [73]:
a = Number(0)
for n in range(1, 100000):
    a = Add(a, Number(n))
e = Evaluator()
e.visit(a)

4999950000

In [74]:
class Evaluator(NodeVisitor):
    ...
    def visit_Add(self, node):
        print('Add:', node)
        lhs = yield node.left
        print('left=', lhs)
        rhs = yield node.right
        print('right=', rhs)
        yield lhs + rhs

这一小节我们演示了生成器和协程在程序控制流方面的强大功能。 避免递归的一个通常方法是使用一个栈或队列的数据结构。 例如，深度优先的遍历算法，第一次碰到一个节点时将其压入栈中，处理完后弹出栈。visit() 方法的核心思路就是这样。

另外一个需要理解的就是生成器中yield语句。当碰到yield语句时，生成器会返回一个数据并暂时挂起。 上面的例子使用这个技术来代替了递归。例如，之前我们是这样写递归：

value = self.visit(node.left)

现在换成yield语句：

value = yield node.left

它会将 node.left 返回给 visit() 方法，然后 visit() 方法调用那个节点相应的 visit_Name() 方法。 yield暂时将程序控制器让出给调用者，当执行完后，结果会赋值给value，

看完这一小节，你也许想去寻找其它没有yield语句的方案。但是这么做没有必要，你必须处理很多棘手的问题。 例如，为了消除递归，你必须要维护一个栈结构，如果不使用生成器，代码会变得很臃肿，到处都是栈操作语句、回调函数等。 实际上，使用yield语句可以让你写出非常漂亮的代码，它消除了递归但是看上去又很像递归实现，代码很简洁。

# 8.23 循环引用数据结构的内存管理

你的程序创建了很多循环引用数据结构(比如树、图、观察者模式等)，你碰到了内存管理难题。

一个简单的循环引用数据结构例子就是一个树形结构，双亲节点有指针指向孩子节点，孩子节点又返回来指向双亲节点。 这种情况下，可以考虑使用 weakref 库中的弱引用。

In [76]:
import weakref

class Node:
    def __init__(self,value):
        self.value = value
        self._parent = None
        self.children = []
        
    def __repr__(self):
        return 'Node({!r:})'.format(self.value)
    
    @property
    def parent(self):
        return None if self._parent is None else self._parent()
    
    @parent.setter
    def parent(self,node):
        self._parent = weakref.ref(node)
        
    def add_child(self,child):
        self.children.append(child)
        child.parent = self

In [77]:
root = Node('parent')

In [78]:
c1 = Node('child')

In [79]:
root.add_child(c1)

In [80]:
print(c1.parent)

Node('parent')


In [81]:
del root

In [82]:
print(c1.parent)

None


循环引用的数据结构在Python中是一个很棘手的问题，因为正常的垃圾回收机制不能适用于这种情形。

In [83]:
class Data:
    def __del__(self):
        print('Data.__del__')
        
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
        
    def add_child(self,child):
        self.children.append(child)
        child.parent = self

In [84]:
a = Data()

In [85]:
del a

Data.__del__


In [86]:
a = Node()

In [87]:
del a

Data.__del__


In [88]:
a = Node()

In [89]:
a.add_child(Node())

In [90]:
del a

可以看到，最后一个的删除时打印语句没有出现。原因是Python的垃圾回收机制是基于简单的引用计数。 当一个对象的引用数变成0的时候才会立即删除掉。而对于循环引用这个条件永远不会成立。 因此，在上面例子中最后部分，父节点和孩子节点互相拥有对方的引用，导致每个对象的引用计数都不可能变成0。

Python有另外的垃圾回收器来专门针对循环引用的，但是你永远不知道它什么时候会触发。 另外你还可以手动的触发它，但是代码看上去很挫

In [91]:
import gc
gc.collect()

Data.__del__
Data.__del__


307922

如果循环引用的对象自己还定义了自己的 \__del__() 方法，那么会让情况变得更糟糕。 假设你像下面这样给Node定义自己的 \__del__() 方法

In [93]:
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        child.parent = self

    # NEVER DEFINE LIKE THIS.
    # Only here to illustrate pathological behavior
    def __del__(self):
        del self.data
        del parent
        del children

这种情况下，垃圾回收永远都不会去回收这个对象的，还会导致内存泄露。 如果你试着去运行它会发现，Data.\__del__ 消息永远不会出现了,甚至在你强制内存回收时

In [94]:
a = Node()

In [95]:
a.add_child(Node())

In [96]:
del a

In [97]:
import gc

In [98]:
gc.collect()

Data.__del__
Data.__del__


Exception ignored in: <function Node.__del__ at 0x0000000005047AE8>
Traceback (most recent call last):
  File "<ipython-input-93-e891a092ce9c>", line 15, in __del__
UnboundLocalError: local variable 'parent' referenced before assignment
Exception ignored in: <function Node.__del__ at 0x0000000005047AE8>
Traceback (most recent call last):
  File "<ipython-input-93-e891a092ce9c>", line 15, in __del__
UnboundLocalError: local variable 'parent' referenced before assignment


28

弱引用消除了引用循环的这个问题，本质来讲，弱引用就是一个对象指针，它不会增加它的引用计数。 你可以通过 weakref 来创建弱引用。

In [99]:
import weakref
a = Node()
a_ref = weakref.ref(a)
a_ref

<weakref at 0x000000000A7E2318; to 'Node' at 0x000000000A7E4710>

为了访问弱引用所引用的对象，你可以像函数一样去调用它即可。如果那个对象还存在就会返回它，否则就返回一个None。 由于原始对象的引用计数没有增加，那么就可以去删除它了

In [100]:
print(a_ref())

In [101]:
del a

Data.__del__


Exception ignored in: <function Node.__del__ at 0x0000000005047AE8>
Traceback (most recent call last):
  File "<ipython-input-93-e891a092ce9c>", line 15, in __del__
UnboundLocalError: local variable 'parent' referenced before assignment


In [102]:
print(a_ref())

None


通过这里演示的弱引用技术，你会发现不再有循环引用问题了，一旦某个节点不被使用了，垃圾回收器立即回收它。

# 8.24 让类支持比较操作

Python类对每个比较操作都需要实现一个特殊方法来支持。 例如为了支持>=操作符，你需要定义一个 \__ge__() 方法。 尽管定义一个方法没什么问题，但如果要你实现所有可能的比较方法那就有点烦人了。

装饰器 functools.total_ordering 就是用来简化这个处理的。 使用它来装饰一个来，你只需定义一个 \__eq__() 方法， 外加其他方法(\__lt__, \__le__, \__gt__, or \__ge__)中的一个即可。 然后装饰器会自动为你填充其它比较方法。

作为例子，我们构建一些房子，然后给它们增加一些房间，最后通过房子大小来比较它们

In [104]:
from functools import total_ordering

class Room:
    def __init__(self,name,length,width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length*self.width
        
@total_ordering
class House:
    def __init__(self,name,style):
        self.name = name 
        self.style = style
        self.rooms = list()
        
    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)
    
    def add_room(self,room):
        self.rooms.append(room)
    
    def str(self):
        return '{}:{} square foot {}'.format(self.name,
                                            self.living_space_footage,
                                            self.style)
    
    def __eq__(self,other):
        return self.living_space_footage == other.living_space_footage
    
    def __lt__(self,other):
        return self.living_space_footage < other.living_space_footage

In [105]:
h1 = House('h1','Cape')

In [106]:
h1.add_room(Room('Master Bedroom',14,21))

In [107]:
h1.add_room(Room('Living Room',18,20))

In [108]:
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))
h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))
h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))

In [109]:
houses = [h1,h2,h3]

In [110]:
print('Is h1 bigger than h2?', h1 > h2)

Is h1 bigger than h2? True


In [111]:
print('Is h2 smaller than h3?', h2 < h3)

Is h2 smaller than h3? True


In [112]:
print('Is h2 greater than or equal to h1?', h2 >= h1)

Is h2 greater than or equal to h1? False


In [113]:
print('Which one is biggest?', max(houses))

Which one is biggest? <__main__.House object at 0x0000000006906470>


In [114]:
print('Which is smallest?', min(houses)) 

Which is smallest? <__main__.House object at 0x0000000006906898>


其实 total_ordering 装饰器也没那么神秘。 它就是定义了一个从每个比较支持方法到所有需要定义的其他方法的一个映射而已。 比如你定义了 __le__() 方法，那么它就被用来构建所有其他的需要定义的那些特殊方法。 实际上就是在类里面像下面这样定义了一些特殊方法：

In [115]:
class House:
    def __eq__(self, other):
        pass
    def __lt__(self, other):
        pass
    # Methods created by @total_ordering
    __le__ = lambda self, other: self < other or self == other
    __gt__ = lambda self, other: not (self < other or self == other)
    __ge__ = lambda self, other: not (self < other)
    __ne__ = lambda self, other: not self == other

# 8.25 创建缓存实例

在创建一个类的对象时，如果之前使用同样参数创建过这个对象， 你想返回它的缓存引用。

这种通常是因为你希望相同参数创建的对象时单例的。 在很多库中都有实际的例子，比如 logging 模块，使用相同的名称创建的 logger 实例永远只有一个。

In [116]:
import logging

In [117]:
a = logging.getLogger('foo')

In [118]:
b = logging.getLogger('bar')

In [119]:
a is b

False

In [122]:
c = logging.getLogger('foo')

In [123]:
a is c

True

为了达到这样的效果，你需要使用一个和类本身分开的工厂函数，

In [124]:
class Spam:
    def __init__(self,name):
        self.name = name
        
import weakref
_spam_cache = weakref.WeakValueDictionary()
def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name]=s
    else:
        s = _spam_cache[name]
    return s

In [125]:
a = get_spam('foo')
b = get_spam('bar')

In [126]:
a is b

False

In [127]:
c = get_spam('foo')

In [128]:
a is c

True

编写一个工厂函数来修改普通的实例创建行为通常是一个比较简单的方法。 但是我们还能否找到更优雅的解决方案呢？

例如，你可能会考虑重新定义类的 __new__() 方法，就像下面这样：

In [129]:
import weakref

class Spam:
    _spam_cache = weakref.WeakValueDictionary()
    def __new__(cls,name):
        if name in cls._spam_cache:
            return cls._spam_cache[name]
        else:
            self = super().__new__(cls)
            cls._spam_cache[name] = self
            return self
        
    def __init__(self,name):
        print('Initializing Spam')
        self.name = name

初看起来好像可以达到预期效果，但是问题是 __init__() 每次都会被调用，不管这个实例是否被缓存了

In [130]:
s = Spam('Dave')

Initializing Spam


In [131]:
t = Spam('Dave')

Initializing Spam


In [132]:
s is t

True

这个或许不是你想要的效果，因此这种方法并不可取。

上面我们使用到了弱引用计数，对于垃圾回收来讲是很有帮助的，关于这个我们在8.23小节已经讲过了。 当我们保持实例缓存时，你可能只想在程序中使用到它们时才保存。 一个 WeakValueDictionary 实例只会保存那些在其它地方还在被使用的实例。 否则的话，只要实例不再被使用了，它就从字典中被移除了。

In [134]:
a = get_spam('foo')
b = get_spam('bar')
c = get_spam('foo')

In [135]:
list(_spam_cache)

['foo', 'bar']

In [136]:
del a

In [137]:
del c

In [138]:
list(_spam_cache)

['bar']

In [139]:
del b

In [140]:
list(_spam_cache)

[]

对于大部分程序而已，这里代码已经够用了。不过还是有一些更高级的实现值得了解下。

首先是这里使用到了一个全局变量，并且工厂函数跟类放在一块。我们可以通过将缓存代码放到一个单独的缓存管理器中：

In [141]:
import weakref

class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()

    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s

    def clear(self):
            self._cache.clear()

class Spam:
    manager = CachedSpamManager()
    def __init__(self, name):
        self.name = name

    def get_spam(name):
        return Spam.manager.get_spam(name)

In [142]:
a = Spam('foo')
b = Spam('foo')

In [143]:
a is b

False

有几种方式可以防止用户这样做，第一个是将类的名字修改为以下划线(_)开头，提示用户别直接调用它。 第二种就是让这个类的 __init__() 方法抛出一个异常，让它不能被初始化

In [144]:
class Spam:
    def __init__(self, *args, **kwargs):
        raise RuntimeError("Can't instantiate directly")

    # Alternate constructor
    @classmethod
    def _new(cls, name):
        self = cls.__new__(cls)
        self.name = name

然后修改缓存管理器代码，使用 Spam._new() 来创建实例，而不是直接调用 Spam() 构造函数：

In [145]:
class CachedSpamManager2:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()

    def get_spam(self, name):
        if name not in self._cache:
            temp = Spam3._new(name)  # Modified creation
            self._cache[name] = temp
        else:
            temp = self._cache[name]
        return temp

    def clear(self):
            self._cache.clear()

class Spam3:
    def __init__(self, *args, **kwargs):
        raise RuntimeError("Can't instantiate directly")

    # Alternate constructor
    @classmethod
    def _new(cls, name):
        self = cls.__new__(cls)
        self.name = name
        return self